# 🧌🗡️ OSRS GOBLIN BOT

## 1. ANNOTATED IMAGES 

The images where anotated on the Roboflow app, making use of their AI helper to aid in the annotations, 
after around 17 manual annotations it was able to annotate the rest of the dataset automatically.

## 2. YOLO TRAINING ON THE ANNOTATED IMAGES

We utilize yolov11n to create the best model which we will then utilize in the streaming part of the project.

In [ ]:
# images uploaded and annotated on the very nice interface that roboflow has, now we import it and download annotations and images on yolov8 format

from roboflow import Roboflow

rf = Roboflow(api_key="")

# Connect to the correct workspace
workspace = rf.workspace("nicolasbojos")

# anually assign the correct project name
project_name = "osrs-goblins-sewlx"

# Load the project correctly
new_project = workspace.project(project_name)

print(f"Project '{project_name}' loaded successfully!")


loading Roboflow workspace...
loading Roboflow project...
Project 'osrs-goblins-sewlx' loaded successfully!


### Checking the versions available on the workspace in order to pick the best one.

In [38]:
versions = new_project.versions()
if not versions:
    print("No dataset versions found. You need to generate one in Roboflow.")
else:
    for version in versions:
        print(f"Available version ID: {version.id}")


Available version ID: nicolasbojos/osrs-goblins-sewlx/4
Available version ID: nicolasbojos/osrs-goblins-sewlx/3
Available version ID: nicolasbojos/osrs-goblins-sewlx/2
Available version ID: nicolasbojos/osrs-goblins-sewlx/1


### Downloading the dataset

In [ ]:
dataset_path = new_project.version(3).download("yolov11")
print(f"Dataset downloaded to: {dataset_path}")

Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to osrs-goblins-3 in yolov11:: 100%|██████████| 326/326 [00:00<00:00, 2053.57it/s]

Dataset downloaded to: <roboflow.core.dataset.Dataset object at 0x7fe13413d840>


### Splitting the dataset

In [ ]:
import os
import shutil
import random

# Define paths
train_images_path = "osrs_goblins_3/train/images"
train_labels_path = "osrs_goblins_3/train/labels"

valid_images_path = "osrs_goblins_3/valid/images"
valid_labels_path = "osrs_goblins_3/valid/labels"

# Ensure valid/ directories exist
os.makedirs(valid_images_path, exist_ok=True)
os.makedirs(valid_labels_path, exist_ok=True)

# Get all images
image_files = [f for f in os.listdir(train_images_path) if f.endswith((".jpg", ".png"))]
num_valid = int(len(image_files) * 0.3)  # Move 30% for validation

print(f" Moved {num_valid} images to validation set.")


 Moved 38 images to validation set.


In [41]:
yaml_content = """train: ./osrs_goblins_3/train/images
val: ./osrs_goblins_3/valid/images

nc: 3
names: ['Goblin', 'Player', 'goblin']
"""

# Save the updated data.yaml
with open("osrs_goblins_3/data.yaml", "w") as f:
    f.write(yaml_content)

print("✅ Updated `data.yaml` with correct paths.")

✅ Updated `data.yaml` with correct paths.


In [ ]:
# check for the classes

with open("osrs_goblins_3/data.yaml", "r") as f:
    print(f.read())

train: ./osrs_goblins_3/train/images
val: ./osrs_goblins_3/valid/images

nc: 3
names: ['Goblin', 'Player', 'goblin']



### Training the model

In [ ]:
!yolo train model=yolo11n.pt data=osrs_goblins_3/data.yaml epochs=50 imgsz=640 batch=16

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=osrs_goblins_3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

### Test

In [ ]:
!yolo predict model=/root/runs/detect/train5/weights/best.pt source="/root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images"

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

image 1/10 /root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images/image26_png.rf.8ee1f6d2eaf1e13393d862ec7ec6cf0a.jpg: 512x640 16 Goblins, 156.4ms
image 2/10 /root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images/image27_png.rf.d003684b8e20a7321a032ca4bd955dfe.jpg: 512x640 20 Goblins, 10.8ms
image 3/10 /root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images/image33_png.rf.5835c25199a3697977a9c76be099d1b7.jpg: 512x640 16 Goblins, 15.4ms
image 4/10 /root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images/image40_png.rf.65e4795736c5f6c5a386df0ce7ed67db.jpg: 512x640 12 Goblins, 10.5ms
image 5/10 /root/Notebooks/computer_viz/MYBOT/osrs_goblins_3/test/images/image46_png.rf.4cb6400f10c1b7eae574ff5c263371af.jpg: 512x640 11 Goblins, 17.8ms
image 6/10 /root/Notebooks/computer_viz/

worked like a charm

![Description](test_image.jpg)


### 3. MAKE THE GAME PLAY ITSELF 

Now that the images are annotated and our model detects the goblins correctly, we need to find a way to extract the frames in a way that it is fast enough and procesable.

Then we need to run the model on said frames and input actions.

# FROM THIS POINT ON WE COULDNT RUN INTERACTIONS WITH WINDOWS FROM LINUX, WE HAD TO MOVE BACK TO WINDOWS.

We will try to find a workaround as using vscode on windows prevents us from using the GPU capabilities of the Local env.

🧌🧌🧌